In [7]:
import warnings
warnings.filterwarnings('ignore')

import warnings
import pandas as pd
import geopandas as gpd
from pathlib import Path

# Suprimir advertencias
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

# Cargar datos geoespaciales de Chile
regiones = gpd.read_file("regiones_SIMP.geojson")
comunas = gpd.read_file("comunas_SIMP.geojson")

def load_geodata(file_path):
    """Carga datos geoespaciales desde un archivo."""
    try:
        return gpd.read_file(file_path)
    except Exception as e:
        raise FileNotFoundError(f"Error al leer el archivo {file_path}: {e}")


def get_user_input():
    """Solicita entrada del usuario para región y comuna."""
    region = input("Indica el nombre de la región (ej: Región Metropolitana de Santiago, Valparaíso, Antofagasta): ").strip()
    comuna = input("Indica el nombre de la comuna (ej: Santiago, Valparaíso, La Serena): ").strip()
    return region, comuna


def filter_region(regiones_gdf, region_nome):
    """Filtra y elimina la región seleccionada de los datos."""
    region_normalizada = region_nome.lower()
    region_encontrada = regiones_gdf[
        regiones_gdf["name"].str.strip().str.lower() == region_normalizada
    ]

    if region_encontrada.empty:
        raise ValueError(f"Región '{region_nome}' no encontrada.")

    otras_regiones = regiones_gdf[
        regiones_gdf["name"].str.strip().str.lower() != region_normalizada
    ]

    return otras_regiones


def process_comunas(comunas_gdf, comuna_nome, region_nome):
    """Procesa las comunas de la región seleccionada, separando la comuna de interés."""
    region_normalizada = region_nome.lower()
    comuna_normalizada = comuna_nome.lower()

    # Filtra solo las comunas de la región indicada
    comunas_da_regiao = comunas_gdf[
        comunas_gdf["region"].str.strip().str.lower() == region_normalizada
    ]

    if comunas_da_regiao.empty:
        raise ValueError(f"Ninguna comuna encontrada para la región '{region_nome}'.")

    # Verifica si la comuna existe en la región
    comuna_selecionada = comunas_da_regiao[
        comunas_da_regiao["name"].str.strip().str.lower() == comuna_normalizada
    ]

    if comuna_selecionada.empty:
        raise ValueError(f"Comuna '{comuna_nome}' no encontrada en la región '{region_nome}'.")

    # Configura la comuna seleccionada
    comuna_selecionada = comuna_selecionada.copy()
    # Elimina el prefijo MUN- si existe, antes de añadir AOI-
    id_limpo = comuna_selecionada["id"].astype(str).str.replace("MUN-", "", regex=False)
    comuna_selecionada["id"] = "AOI-" + id_limpo
    comuna_selecionada["poly_type"] = "core"

    # Otras comunas de la misma región (excepto la seleccionada)
    outras_comunas_regiao = comunas_da_regiao[
        comunas_da_regiao["name"].str.strip().str.lower() != comuna_normalizada
    ]

    return comuna_selecionada, outras_comunas_regiao


def create_project(outras_regiones, outras_comunas_regiao, comuna_selecionada):
    """Crea el proyecto final concatenando todas las capas."""
    return pd.concat([outras_regiones, outras_comunas_regiao, comuna_selecionada],
                    ignore_index=True)


def save_project(projeto_gdf, comuna_nome):
    """Guarda el proyecto en un archivo GeoJSON."""
    output_filename = f"{comuna_nome.replace(' ', '_')}_AOI.geojson"
    try:
        projeto_gdf.to_file(output_filename, driver='GeoJSON')
        print(f"Proyecto guardado con éxito en {output_filename}")
    except Exception as e:
        raise IOError(f"Error al guardar el proyecto: {e}")


def main():
    """Función principal del programa."""
    try:
        region, comuna = get_user_input()

        regiones_data = regiones.copy()  
        comunas_data = comunas.copy()

        outras_regiones = filter_region(regiones_data, region)
        comuna_selecionada, outras_comunas_regiao = process_comunas(comunas_data, comuna, region)

        projeto = create_project(outras_regiones, outras_comunas_regiao, comuna_selecionada)
        save_project(projeto, comuna)

    except (FileNotFoundError, ValueError, IOError) as e:
        print(f"Error: {e}")
    except KeyboardInterrupt:
        print("\nOperación cancelada por el usuario.")
    except Exception as e:
        print(f"Error inesperado: {e}")


if __name__ == "__main__":
    main()

Indica el nombre de la región (ej: Región Metropolitana de Santiago, Valparaíso, Antofagasta):  Región Metropolitana de Santiago
Indica el nombre de la comuna (ej: Santiago, Valparaíso, La Serena):  Santiago


Proyecto guardado con éxito en Santiago_AOI.geojson


In [2]:
teste = gpd.read_file("Santiago_AOI.geojson")

In [5]:
teste.to_file("Santiago_AOI.geojson")

In [ ]:
comunas 